In [1]:
import pandas as pd
from src.utils import parallel_match_organizations, treat_string

In [2]:
# Example usage
meetings_df = pd.read_csv("./data/meetings.csv", dtype={"member_id": str})
register_df = pd.read_excel(
    r"D:\OneDrive\UFPR\00.PPGCP\00. Doutorado\Tese\Base de dados\Transparency register\Organisations in Transparency Register-2024-JAN.xls"
)

# Explode the attendees column and create new rows for each attendee
meetings_df = meetings_df.reset_index(drop=True)  # Reset index to avoid duplicates
meetings_df = meetings_df.assign(
    attendees=meetings_df['attendees'].str.split("|")
).explode('attendees')

In [3]:
# Match organizations
matched_df = parallel_match_organizations(
    meetings_df,
    register_df,
    meetings_col='attendees',
    register_col='Name',
    batch_size=10,
    threshold=75.0  # Adjust threshold as needed
)

# Filter high-confidence matches
high_confidence_matches = matched_df[matched_df['match_confidence'] >= 85]
high_confidence_matches.to_csv("high_confidence_matches.csv")

Processing batches: 100%|██████████| 9937/9937 [5:40:13<00:00,  2.05s/it]  


In [4]:
from thefuzz import fuzz

treated_name = treat_string("National Institute of Standards and Technology")
treated_candidate = treat_string("Institute of Metals and Technology")
# Try different fuzzy matching algorithms
ratio = fuzz.ratio(treated_name, treated_candidate)
partial_ratio = fuzz.partial_ratio(treated_name, treated_candidate)
token_sort_ratio = fuzz.token_sort_ratio(treated_name, treated_candidate)
token_set_ratio = fuzz.token_set_ratio(treated_name, treated_candidate)

print(treated_name,'\n', treated_candidate, '\n', ratio, partial_ratio, token_sort_ratio, token_set_ratio)

national institute of standards and technology 
 institute of metals and technology 
 78 82 78 89


In [37]:
matched_df[matched_df['match_confidence'] <= 85].sample(10)[['attendees', 'matched_organization', 'match_confidence']]

,attendees,matched_organization,match_confidence
1296,Kreisjugendring Pinneberg,None,0.0
160,Kansalaistapaaminen,None,0.0
3169,Assistant United States Trade Representative f...,None,0.0
2171,Hagainitiativet,None,0.0
2473,Business Oulu,None,0.0
369,Arbeiterkammer AK EUROPA,None,0.0
2090,Deputy mayor of Uhersky Brod,None,0.0
3971,Community Plant Variety Office,None,0.0
4589,"Verband der Getreide-, Mühlen- und Stärkewirts...",None,0.0
1524,IRU,None,0.0


In [41]:
print(len(high_confidence_matches))

2166
